# 建議先從rag-faiss.ipynb開始跑 環境最複雜 但是弄完 4份都可以跑

事前準備:

python 3

VScode商城:

python

jupyter

import和拿key

沒裝openai_SDK使用:```pip install openai```

沒裝tabulate使用:```pip install tabulate```

沒設定key使用:```setx OPENAI_API_KEY “<yourkey>”```

```echo %OPENAI_API_KEY%```可以看你的KEY

In [26]:
import os
from openai import OpenAI
client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')

# input在此輸入

In [27]:
input = "hey flora! Im having some issue using Instagram! can you teach me how to add friend?"

創assistant,thread,message!

In [28]:
from tabulate import tabulate

# Upload a file with an "assistants" purpose
file = client.files.create(
    file=open("data\manual.csv", "rb"),
    purpose='assistants'
)

assistant = client.beta.assistants.create(
    name="Flora Customer Service",
    instructions="You are customer service from Flora.You can only answer questions related to Flora. when answering any question, you must always answer base on manual.csv which user have provided and respond only once.",
    tools=[{"type": "retrieval"},{"type": "code_interpreter"}],
    model="gpt-3.5-turbo-1106",
    file_ids=[file.id]
)

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Can you find me the answer according to\""+input+"\"?"
)


沒裝tabulate很醜

tabulate文件:https://pypi.org/project/tabulate/

表示assistant,thread,message內容

In [29]:
print(tabulate(assistant,headers=['assistant','content'],tablefmt="simple_outline"))

┌──────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ assistant    │ content                                                                                                                                                                                                    │
├──────────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ id           │ asst_h3EVM0v8XCF4bCvuINhHRO0M                                                                                                                                                                              │
│ created_at   │ 1707104014                                                                                     

In [30]:
print(tabulate(thread,headers=['thread','content'],tablefmt="simple_outline"))

┌────────────┬─────────────────────────────────┐
│ thread     │ content                         │
├────────────┼─────────────────────────────────┤
│ id         │ thread_5Jii6obsOglgSLS7I5L9wfkK │
│ created_at │ 1707104014                      │
│ metadata   │ {}                              │
│ object     │ thread                          │
└────────────┴─────────────────────────────────┘


In [31]:
print(tabulate(message,headers=['message','content'],tablefmt="simple_outline"))

┌──────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ message      │ content                                                                                                                                                                                              │
├──────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ id           │ msg_daPDAEDBLMjv2oqxf1dg56O4                                                                                                                                                                         │
│ assistant_id │                                                                                                                        

這次的run

可以使用新instruction複寫assisstant裡的instruction!

like:

```
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)
```


In [32]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

查看run的狀態!

以上如果還沒跑完會顯示queued!

以上如果跑完會顯示completed!

In [33]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print(tabulate(run,headers=['run','content'],tablefmt='simple_outline'))

┌─────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ run             │ content                                                                                                                                                                                                    │
├─────────────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ id              │ run_MC9ISturN4pWf6H7c9WWzjRg                                                                                                                                                                               │
│ assistant_id    │ asst_h3EVM0v8XCF4bCvuINhHRO0M                                                   

跑完後打開查看history messages!

In [38]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id,
  order='asc'
)
for i in messages.data:
    print(tabulate(i,tablefmt='simple_outline'))

┌──────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ id           │ msg_daPDAEDBLMjv2oqxf1dg56O4                                                                                                                                                                         │
│ assistant_id │                                                                                                                                                                                                      │
│ content      │ [MessageContentText(text=Text(annotations=[], value='Can you find me the answer according to"hey flora! Im having some issue using Instagram! can you teach me how to add friend?"?'), type='text')] │
│ created_at   │ 1707104014                                                                                                             

最後一個訊息顯示

In [40]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
print(messages.data[0].content[0].text.value)

I'm sorry, but it seems that the provided file does not contain information on how to add a friend on Instagram. If there's any specific section in the file that you would like me to review, please let me know and I can assist further.
